In [77]:
!pip install emoji

In [78]:
import emoji
import nltk
import pandas as pd
import re
import requests
import spacy

from nltk.corpus import stopwords

In [79]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Data Uploading

In [81]:
url = "https://raw.githubusercontent.com/SaraHoxha/emotion-detection-txa/refs/heads/main/Data%20Collection/Reddit_data.csv?token=GHSAT0AAAAAAC2L3OWURVBSJZBPTIZADQ6GZZ7OCRQ"
rdt = pd.read_csv(url)

## Text Cleaning

In [82]:
rdt = rdt.rename(columns = {'text': 'Text'})
rdt['created_utc'] = pd.to_datetime(rdt['created_utc'], unit = 's', errors = 'coerce')
rdt['num_comments'] = pd.to_numeric(rdt['num_comments'], errors = 'coerce').fillna(0).astype(int)

In [83]:
def clean_text(post):
    if not isinstance(post, str):
        return ""

    post = post.lower() # Convert to lowercase
    post = re.sub(r'http\S+|www.\S+', '', post) # Remove links
    post = re.sub(r'Edit[0-9]*:', '', post, flags=re.IGNORECASE) # Remove edit chunks
    post = re.sub(r'(.)\1{2,}', r'\1\1', post) # Normalize words with repeated letters
    post = re.sub(r"[^a-zA-Z\s!\"'\?\$\€\£]+", '', post) # Keep only alphabetic and relevant special characters
    post = re.sub(r'\s+', ' ', post).strip()  # Remove unecessary spaces

    return post

In [84]:
rdt['clean_text'] = rdt['Text'].apply(clean_text)

In [85]:
rdt = rdt.rename(columns = {'clean_text': 'text'}).drop(columns = ['Text'])
rdt = rdt[rdt['text'] != '']

In [ ]:
rdt.head(10)

,topic_title,id,author,subreddit,link_id,parent_id,num_comments,upvotes,created_utc,text
0,Oh how offices have changed,1fy22yp,naixelsyd,remotework,1fy22yp,NaN,435,4911,2024-10-07 07:32:09,for those who don't know years ago commuting i...
1,Oh how offices have changed,lqqyui9,fejobelo,remotework,1fy22yp,t3_1fy22yp,0,581,2024-10-07 08:33:42,back in the day every time we stayed working a...
2,Oh how offices have changed,lqrf0ak,TrekJaneway,remotework,1fy22yp,t3_1fy22yp,0,138,2024-10-07 11:27:56,and a pensiondont forget a pension but that wa...
3,Oh how offices have changed,lqrjvzl,CZ1988_,remotework,1fy22yp,t3_1fy22yp,0,69,2024-10-07 12:08:10,years ago was still a cube farm you did get yo...
4,Oh how offices have changed,lqrrh6n,Ok-Combination-5201,remotework,1fy22yp,t3_1fy22yp,0,17,2024-10-07 13:03:07,the movie office space came out years ago that...
5,Oh how offices have changed,lqrg9h3,Amphrael,remotework,1fy22yp,t3_1fy22yp,0,79,2024-10-07 11:38:45,when i was in the work force years ago few peo...
6,Oh how offices have changed,lqrgmzs,Vendevende,remotework,1fy22yp,t3_1fy22yp,0,66,2024-10-07 11:41:54,you were also sexually harassed quite a bit if...
7,Oh how offices have changed,lqspb66,InevitableCodeRedo,remotework,1fy22yp,t3_1fy22yp,0,11,2024-10-07 16:17:46,overwhelmingly most of us had none of those th...
8,Oh how offices have changed,lqrguea,lai4basis,remotework,1fy22yp,t3_1fy22yp,0,28,2024-10-07 11:43:37,this isn't even close to accurate
9,Oh how offices have changed,lqrl5w7,DogOrDonut,remotework,1fy22yp,t3_1fy22yp,0,8,2024-10-07 12:18:00,the only part of this that is true is the flex...


## Variables Adding and Transformtions

In [86]:
rdt = rdt[rdt['text'].str.len() > 30]
rdt.head(10)

,topic_title,id,author,subreddit,link_id,parent_id,num_comments,upvotes,created_utc,text
0,Oh how offices have changed,1fy22yp,naixelsyd,remotework,1fy22yp,NaN,435,4911,2024-10-07 07:32:09,for those who don't know years ago commuting i...
1,Oh how offices have changed,lqqyui9,fejobelo,remotework,1fy22yp,t3_1fy22yp,0,581,2024-10-07 08:33:42,back in the day every time we stayed working a...
2,Oh how offices have changed,lqrf0ak,TrekJaneway,remotework,1fy22yp,t3_1fy22yp,0,138,2024-10-07 11:27:56,and a pensiondont forget a pension but that wa...
3,Oh how offices have changed,lqrjvzl,CZ1988_,remotework,1fy22yp,t3_1fy22yp,0,69,2024-10-07 12:08:10,years ago was still a cube farm you did get yo...
4,Oh how offices have changed,lqrrh6n,Ok-Combination-5201,remotework,1fy22yp,t3_1fy22yp,0,17,2024-10-07 13:03:07,the movie office space came out years ago that...
5,Oh how offices have changed,lqrg9h3,Amphrael,remotework,1fy22yp,t3_1fy22yp,0,79,2024-10-07 11:38:45,when i was in the work force years ago few peo...
6,Oh how offices have changed,lqrgmzs,Vendevende,remotework,1fy22yp,t3_1fy22yp,0,66,2024-10-07 11:41:54,you were also sexually harassed quite a bit if...
7,Oh how offices have changed,lqspb66,InevitableCodeRedo,remotework,1fy22yp,t3_1fy22yp,0,11,2024-10-07 16:17:46,overwhelmingly most of us had none of those th...
8,Oh how offices have changed,lqrguea,lai4basis,remotework,1fy22yp,t3_1fy22yp,0,28,2024-10-07 11:43:37,this isn't even close to accurate
9,Oh how offices have changed,lqrl5w7,DogOrDonut,remotework,1fy22yp,t3_1fy22yp,0,8,2024-10-07 12:18:00,the only part of this that is true is the flex...


In [88]:
rdt['complete_text'] = rdt['topic_title'] + ' '+  rdt['text']
rdt = rdt[['subreddit', 'link_id', 'parent_id', 'complete_text', 'created_utc', 'upvotes']]
rdt = rdt.rename(columns = {'complete_text': 'text'})

rdt.head(10)

,subreddit,link_id,parent_id,text,created_utc,upvotes
0,remotework,1fy22yp,NaN,Oh how offices have changed for those who don'...,2024-10-07 07:32:09,4911
1,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed back in the day ev...,2024-10-07 08:33:42,581
2,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed and a pensiondont ...,2024-10-07 11:27:56,138
3,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed years ago was stil...,2024-10-07 12:08:10,69
4,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed the movie office s...,2024-10-07 13:03:07,17
5,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed when i was in the ...,2024-10-07 11:38:45,79
6,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed you were also sexu...,2024-10-07 11:41:54,66
7,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed overwhelmingly mos...,2024-10-07 16:17:46,11
8,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed this isn't even cl...,2024-10-07 11:43:37,28
9,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed the only part of t...,2024-10-07 12:18:00,8


In [ ]:
rdt = rdt[rdt['upvotes'] != 0]

,subreddit,link_id,parent_id,text,created_utc,upvotes
0,remotework,1fy22yp,NaN,Oh how offices have changed for those who don'...,2024-10-07 07:32:09,4911
1,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed back in the day ev...,2024-10-07 08:33:42,581
2,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed and a pensiondont ...,2024-10-07 11:27:56,138
3,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed years ago was stil...,2024-10-07 12:08:10,69
4,remotework,1fy22yp,t3_1fy22yp,Oh how offices have changed the movie office s...,2024-10-07 13:03:07,17
...,...,...,...,...,...,...
175142,remotejobs,va9o1y,t3_va9o1y,Remote jobs that are ESSENTIAL to infrastructu...,2022-06-12 04:02:05,3
175143,remotejobs,va9o1y,t3_va9o1y,Remote jobs that are ESSENTIAL to infrastructu...,2022-06-12 18:01:16,1
175144,remotejobs,va9o1y,t3_va9o1y,Remote jobs that are ESSENTIAL to infrastructu...,2022-06-22 18:22:45,1
175145,remotejobs,va9o1y,t1_ic414vi,Remote jobs that are ESSENTIAL to infrastructu...,2022-06-14 15:24:38,2


## Removing Emojis

In [93]:
def remove_emojis(text):
    return emoji.replace_emoji(text, replace = '')

In [94]:
rdt['text'] = rdt['text'].apply(lambda x: remove_emojis(x))

C:\Users\Usuario\AppData\Local\Temp\ipykernel_25912\2674499097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdt['text'] = rdt['text'].apply(lambda x: remove_emojis(x))


## Special Characters Mapping


In [96]:
def map_special_chars(text):

    text = text.replace("!", " exclamation")
    text = text.replace("?", " question")
    text = text.replace("$", " money")
    text = text.replace("€", " money")
    text = text.replace("£", " money")
    text = text.replace("%", " percentage")

    return text

In [100]:
rdt['mapped_text'] = rdt['text'].apply(map_special_chars)
rdt = rdt.drop(columns = ['text'])
rdt = rdt.rename(columns = {'mapped_text': 'text'})

C:\Users\Usuario\AppData\Local\Temp\ipykernel_25912\2132209729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdt['mapped_text'] = rdt['text'].apply(map_special_chars)


In [102]:
rdt.to_csv('prep_df_final.csv', index = False)

In [103]:
# Calculate the length of each text
text_lengths = rdt['text'].str.len()
rdt['charnum'] = text_lengths

# Find the 91th percentile (threshold for top 9%)
percentile_91 = text_lengths.quantile(0.906)

# Get the rows that are in the top 9%
long_texts = text_lengths[text_lengths >= percentile_91]

print(f"Character count threshold (91th percentile): {percentile_91:.0f}")
print(f"Number of texts above threshold: {len(long_texts)}")
print(f"Average length of long texts: {long_texts.mean():.0f} characters")

Character count threshold (91th percentile): 632
Number of texts above threshold: 13441
Average length of long texts: 1027 characters


In [104]:
long_texts = rdt[rdt['charnum'] > percentile_91]
short_texts = rdt[rdt['charnum'] <= percentile_91]
# Count Reddit rows in long_texts
total_count = len(long_texts)

print(f"Number of Reddit posts: {total_count}")
print(f"Percentage of Reddit posts: {(total_count/total_count)*100:.1f}%")

Number of Reddit posts: 13380
Percentage of Reddit posts: 100.0%


In [105]:
del rdt['charnum']

In [106]:
rdt

,subreddit,link_id,parent_id,created_utc,upvotes,text
0,remotework,1fy22yp,NaN,2024-10-07 07:32:09,4911,Oh how offices have changed for those who don'...
1,remotework,1fy22yp,t3_1fy22yp,2024-10-07 08:33:42,581,Oh how offices have changed back in the day ev...
2,remotework,1fy22yp,t3_1fy22yp,2024-10-07 11:27:56,138,Oh how offices have changed and a pensiondont ...
3,remotework,1fy22yp,t3_1fy22yp,2024-10-07 12:08:10,69,Oh how offices have changed years ago was stil...
4,remotework,1fy22yp,t3_1fy22yp,2024-10-07 13:03:07,17,Oh how offices have changed the movie office s...
...,...,...,...,...,...,...
175142,remotejobs,va9o1y,t3_va9o1y,2022-06-12 04:02:05,3,Remote jobs that are ESSENTIAL to infrastructu...
175143,remotejobs,va9o1y,t3_va9o1y,2022-06-12 18:01:16,1,Remote jobs that are ESSENTIAL to infrastructu...
175144,remotejobs,va9o1y,t3_va9o1y,2022-06-22 18:22:45,1,Remote jobs that are ESSENTIAL to infrastructu...
175145,remotejobs,va9o1y,t1_ic414vi,2022-06-14 15:24:38,2,Remote jobs that are ESSENTIAL to infrastructu...


In [107]:
rdt.to_csv('prep_df_final_91.csv', index = False)